In [40]:
import cv2
import itertools
import numpy as np
import mediapipe as mp
import pandas as pd

In [41]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    refine_landmarks=True,
    static_image_mode=True,
    max_num_faces = 1,
)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

for i in range(100):
    image = cv2.imread("./dataset/train/img/%s.png"%i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    globals()['height'+str(i)] = image.shape[0]
    
    results = face_mesh.process(image)
    
    for single_face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list = single_face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec = drawing_spec,            
        )
    
    INDEX1 = mp_face_mesh.FACEMESH_LEFT_EYE
    INDEX2 = mp_face_mesh.FACEMESH_RIGHT_EYE
    
    INDEX_LIST1 = list(itertools.chain(*INDEX1))
    INDEX_LIST2 = list(itertools.chain(*INDEX2))
    INDEX_LIST = INDEX_LIST1 + INDEX_LIST2
    
    globals()['facemesh'+str(i)] = np.zeros((len(INDEX_LIST), 2))
    
    for single_face_landmarks in results.multi_face_landmarks:
        for j, k in enumerate(INDEX_LIST):
            coordinates = single_face_landmarks.landmark[k]
            globals()['facemesh'+str(i)][j][0] = coordinates.x * image.shape[1]
            globals()['facemesh'+str(i)][j][1] = coordinates.y * image.shape[0]

In [42]:
len(INDEX_LIST)

64

In [43]:
for i in range(100):
    land = np.zeros((len(facemesh0),2))
    
    for j in range(len(facemesh0)):
        land[j][0] = globals()['facemesh'+str(i)][j][0]
        land[j][1] = globals()['facemesh'+str(i)][j][1]
        
    x = land[:, 0]
    y = land[:, 1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))
    
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [44]:
train_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        train_x[i][j] = globals()['norm'+str(i)][j]

In [45]:
for i in range(100):
    image = cv2.imread("./dataset/test/img/%s.png"%i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    globals()['height'+str(i)] = image.shape[0]
    
    results = face_mesh.process(image)
    
    for single_face_landmark in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list = single_face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec = drawing_spec,            
        )
    
    INDEX1 = mp_face_mesh.FACEMESH_LEFT_EYE
    INDEX2 = mp_face_mesh.FACEMESH_RIGHT_EYE
    
    INDEX_LIST1 = list(itertools.chain(*INDEX1))
    INDEX_LIST2 = list(itertools.chain(*INDEX2))
    INDEX_LIST = INDEX_LIST1 + INDEX_LIST2
    
    globals()['facemesh'+str(i)] = np.zeros((len(INDEX_LIST), 2))
    
    for single_face_landmarks in results.multi_face_landmarks:
        for j, k in enumerate(INDEX_LIST):
            coordinates = single_face_landmarks.landmark[k]
            globals()['facemesh'+str(i)][j][0] = coordinates.x * image.shape[1]
            globals()['facemesh'+str(i)][j][1] = coordinates.y * image.shape[0]

In [46]:
for i in range(100):
    land = np.zeros((len(facemesh0),2))
    
    for j in range(len(facemesh0)):
        land[j][0] = globals()['facemesh'+str(i)][j][0]
        land[j][1] = globals()['facemesh'+str(i)][j][1]
        
    x = land[:, 0]
    y = land[:, 1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))
    
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [47]:
test_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        test_x[i][j] = globals()['norm'+str(i)][j]

In [48]:
train_y = pd.read_csv('./dataset/train/train.csv')
train_y = train_y.iloc[:, 15:21]
test_y = pd.read_csv('./dataset/test/test.csv')
test_y = test_y.iloc[:, 15:21]

In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(train_x, train_y)

LinearRegression()

In [50]:
y_pred = model.predict(test_x)
mae = mean_absolute_error(test_y, y_pred)
mae

0.14312007016367048

In [51]:
pred = pd.DataFrame(y_pred)
pred.columns = ['eyeSize','eyePosition', 'eyeDepth','eyeRotation','eyeDistance','eyeSquint']
pred

,eyeSize,eyePosition,eyeDepth,eyeRotation,eyeDistance,eyeSquint
0,0.052021,0.297061,0.541088,0.666056,0.474412,0.383438
1,-0.112066,0.186681,0.052106,0.734961,0.602031,0.300383
2,0.527168,0.697480,0.625724,0.724574,0.716522,0.450505
3,0.760694,0.683587,0.630631,0.629672,0.268802,0.479327
4,-0.267986,0.748527,0.609368,0.544047,0.082815,0.446746
...,...,...,...,...,...,...
95,0.169382,0.729884,0.312475,0.310698,0.172697,0.246416
96,0.418198,0.604065,0.484970,0.872786,0.632334,0.565897
97,0.798407,0.292979,0.296358,0.436182,0.781077,0.527958
98,0.384860,0.212822,0.378491,0.554916,0.361513,0.465586


In [52]:
test_y

,earsSize,eyeSize,eyePosition,eyeDepth,eyeRotation,eyeDistance
0,0.369729,0.439447,0.758647,0.370812,0.506665,0.462228
1,0.351753,0.235531,0.306835,0.310255,0.799276,0.280601
2,0.390302,0.728159,0.711342,0.489252,0.768084,0.467365
3,0.629453,0.647029,0.614792,0.624584,0.250405,0.448013
4,0.706109,0.763835,0.662353,0.226584,0.220240,0.569739
...,...,...,...,...,...,...
95,0.384638,0.747057,0.606980,0.688712,0.295096,0.276458
96,0.663839,0.757262,0.391804,0.574604,0.728450,0.522360
97,0.334792,0.345698,0.455230,0.572820,0.659950,0.482190
98,0.789721,0.334341,0.318302,0.478478,0.285375,0.471584
